# Determining IHP Funding for One County Disasters

## Quick Summary
An OpenFEMA user wanted to determine the rank of Individuals and Households Program (IHP) funding for only those disasters that were declared for one county. Output was to include the disaster number, state, declaration type, declaration date, declaration title, and Individual Assistance (IA) funding:

    DR-4676-IL 
    Declared 10/14/2022
    Severe Storms & Flooding
    IA Counties: 1
    IA Funding: $27,419,277.04

Generating this summary from OpenFEMA data is easily accomplished with the following high-level steps:

- Download the [Disaster Declarations Summaries - V2](https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2) to get disasters by county or place
- Download the [FEMA Web Disaster Summaries - V1](https://www.fema.gov/openfema-data-page/fema-web-disaster-summaries-v1) to get financial information for the disasters
- Exclude those disasters that have a designatedArea of “Statewide”. Be mindful of fipsCountyCode = “000” where the designatedArea is not “Statewide” – these are most likely tribal areas.
- Aggregate by fipsCountyCode (or placeCode), counting the counties and limiting the return to just those containing 1 county
- Join the remaining records with the FEMA Web Disaster Summaries data to get the financial information

## Get the appropriate disaster information
As a rule of thumb, try and utilize the API to filter out as much unnecessary data as possible and return only what is needed. The following code excludes Statewide and Tribal declarations and those that do not have IA declared. This limits the returned records from about 65K to about 24K.

<div class="alert alert-block alert-warning">
    <b>Note:</b> The analysis presented in this notebook has not been vetted by the IA Program Office. It is presented for illustrative purposes.
</div>

<div class="alert alert-block alert-warning">
    <b>Note:</b> This is not meant to be a Python language tutorial. The point is to show how to utilize the OpenFEMA API in Python to summarize information. There are other, more Pythonic ways that this can be done. If you are writing production quality code, it is recommended that you follow industry best practices - replace embedded constants, evaluate returned values, add error/exception handling, add logging, proper object cleanup, build for resilience by adding retries if failure, etc.
</div>

In [12]:
# declare a URL handling module
import urllib.request
import json

# define URL for the FEMA Web Disaster Declarations endpoint
baseUrl = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?"

# exclude statewide and tribal disasters and include only those with IA declared
sFilter = "$filter=fipsCountyCode%20ne%20%27000%27%20and%20(ihProgramDeclared%20eq%20true%20or%20iaProgramDeclared%20eq%20true)"

# only return what we might need
sSelect = "&$select=disasterNumber,state,designatedArea,declarationType,declarationDate,declarationTitle,fipsCountyCode,placeCode"

# let's sort, turn off metadata, grab all records, return format in jsona because it is easier to turn into a dataframe
sOther = "&$orderby=disasterNumber,state&$top=0&$metadata=false&$format=jsona"

# the final URL can be pasted into a browser to return the same set of data
print(baseUrl + sFilter + sSelect + sOther) 

# open the URL as defined above and create a request object 
request = urllib.request.urlopen(baseUrl + sFilter + sSelect + sOther)

# actually read the data and transform to Python dictionary
result = request.read()
jsonData = json.loads(result.decode('utf-8'))

# return a count of records read
len(jsonData)


https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fipsCountyCode%20ne%20%27000%27%20and%20(ihProgramDeclared%20eq%20true%20or%20iaProgramDeclared%20eq%20true)&$select=disasterNumber,state,designatedArea,declarationType,declarationDate,declarationTitle,fipsCountyCode,placeCode&$orderby=disasterNumber,state&$top=0&$metadata=false&$format=jsona


24037

## Summarize/aggregate data to count counties by disaster
To make it easier to perform summary operations, we will convert the JSON data returned above into a Pandas dataframe structure. Libraries that provide similar capabilities exist, but Pandas is one of the most common. This will let us summarize, filter, and perform all kinds of operations on our data.

Once we have a dataframe, we can aggregate by disaster and state, counting each county.

In [14]:
import pandas as pd

# save as a dataframe for easier analysis
df = pd.DataFrame(jsonData)

# use pandas dataframe groupby and aggregation features
dfSumByDstr = df.groupby(
    ['disasterNumber',  'state']
).agg(
    # count number of counties, but return other data
    state=('state', 'first'),
    decArea=('designatedArea', 'first'),
    decType=('declarationType', 'first'),
    decDate=('declarationDate', 'first'),
    decTitle=('declarationTitle', 'first'),
    CountyCount=('fipsCountyCode', 'count')
)

dfSumByDstr

,,state,decArea,decType,decDate,decTitle,CountyCount
disasterNumber,state,,,,,,
91,IN,IN,Clay (County),DR,1959-01-29T00:00:00.000Z,FLOOD,1
183,CA,CA,Alpine (County),DR,1964-12-24T00:00:00.000Z,HEAVY RAINS & FLOODING,31
184,OR,OR,Polk (County),DR,1964-12-24T00:00:00.000Z,HEAVY RAINS & FLOODING,36
185,WA,WA,Pacific (County),DR,1964-12-29T00:00:00.000Z,HEAVY RAINS & FLOODING,21
186,ID,ID,Elmore (County),DR,1964-12-31T00:00:00.000Z,HEAVY RAINS & FLOODING,27
...,...,...,...,...,...,...,...
4727,MS,MS,Jackson (County),DR,2023-08-12T00:00:00.000Z,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNAD...",2
4728,IL,IL,Cook (County),DR,2023-08-15T00:00:00.000Z,SEVERE STORMS AND FLOODING,1
4730,AK,AK,Lower Kuskokwim Regional Educational Attendanc...,DR,2023-08-23T00:00:00.000Z,FLOODING,5


## Filter such that we only retain those that apply to one county

In [15]:
# only those with 1 county
dfOneCounty = dfSumByDstr.loc[dfSumByDstr["CountyCount"] == 1]
dfOneCounty

,,state,decArea,decType,decDate,decTitle,CountyCount
disasterNumber,state,,,,,,
91,IN,IN,Clay (County),DR,1959-01-29T00:00:00.000Z,FLOOD,1
212,CA,CA,Humboldt (County),DR,1966-01-22T00:00:00.000Z,SEVERE STORMS & FLOODING,1
222,TX,TX,Hudspeth (County),DR,1966-09-27T00:00:00.000Z,HEAVY RAINS & FLOODING,1
244,TX,TX,Hill (County),DR,1968-06-10T00:00:00.000Z,HEAVY RAINS & FLOODING,1
250,MN,MN,Roseau (County),DR,1968-09-09T00:00:00.000Z,HEAVY RAINS & FLOODING,1
...,...,...,...,...,...,...,...
4676,IL,IL,St. Clair (County),DR,2022-10-14T00:00:00.000Z,SEVERE STORM AND FLOODING,1
4709,FL,FL,Broward (County),DR,2023-04-27T00:00:00.000Z,"SEVERE STORMS, TORNADOES, AND FLOODING",1
4715,GU,GU,Guam (County-equivalent),DR,2023-05-25T00:00:00.000Z,TYPHOON MAWAR,1


## Get IA financial information by disaster
We will use the same technique as above to get financial data using the OpenFEMA API. We can limit records by ignoring those that do not contain any IA/IHP funding. We will return only a couple of fields.


In [16]:
# define URL for the FEMA Web Disaster Declarations endpoint
baseUrl = "https://www.fema.gov/api/open/v1/FemaWebDisasterSummaries?"

# exclude statewide and tribal disasters and include only those with IA declared
sFilter = "$filter=totalAmountIhpApproved%20ne%20null"

# only return what we might need
sSelect = "&$select=disasterNumber,totalAmountIhpApproved,totalNumberIaApproved"

# let's sort, get all records, exclude metadata, return as JSON array
sOther = "&$orderby=disasterNumber&$top=0&$metadata=false&$format=jsona"

# open the URL as defined above and create a request object, read the data, and transform to Python dict
request = urllib.request.urlopen(baseUrl + sFilter + sSelect + sOther)
result = request.read()
jsonData = json.loads(result.decode('utf-8'))

# save as a dataframe for easier analysis
dfFinancials = pd.DataFrame(jsonData)

# large dollar amounts will be displayed with scientific notation unless we suppress as follows
pd.options.display.float_format = '{:.0f}'.format
dfFinancials

,disasterNumber,totalAmountIhpApproved,totalNumberIaApproved
0,1439,57974742,19827
1,1440,470372,272
2,1441,1880461,412
3,1442,3306085,530
4,1443,2028549,555
...,...,...,...
518,4727,2357153,611
519,4728,190344802,54897
520,4730,2071716,218
521,4734,62303439,30713


## Join the financial and disaster data
We must join these two datasets for our final answer. Sort from highest total amount to lowest. Also, save the results to an excel file.

In [17]:
# join with financial information
dfFinal = dfOneCounty.merge(dfFinancials.set_index('disasterNumber'), on='disasterNumber').sort_values(by = 'totalAmountIhpApproved', ascending = False)

# for better viewing, remove the time component from the declaration date
dfFinal['decDate'] = dfFinal['decDate'].str[:10]

# export to a file (file will save to the directory where the code is run unless the path is provided)
dfFinal.to_excel('ihp_funding_one_county_disasters.xlsx')

# display
dfFinal


,state,decArea,decType,decDate,decTitle,CountyCount,totalAmountIhpApproved,totalNumberIaApproved
disasterNumber,,,,,,,,
4728,IL,Cook (County),DR,2023-08-15,SEVERE STORMS AND FLOODING,1,190344802,54897
1446,GU,Guam (County-equivalent),DR,2002-12-08,SUPER TYPHOON PONGSONA,1,125463061,20699
4715,GU,Guam (County-equivalent),DR,2023-05-25,TYPHOON MAWAR,1,68617743,19754
4709,FL,Broward (County),DR,2023-04-27,"SEVERE STORMS, TORNADOES, AND FLOODING",1,35995702,9572
4724,HI,Maui (County),DR,2023-08-10,WILDFIRES,1,28881643,6148
4676,IL,St. Clair (County),DR,2022-10-14,SEVERE STORM AND FLOODING,1,27421965,6098
4235,MP,Saipan (Municipality),DR,2015-08-05,TYPHOON SOUDELOR,1,25097246,4864
1855,KY,Jefferson (County),DR,2009-08-14,"SEVERE STORMS, STRAIGHT-LINE WINDS AND FLOODING",1,17693444,7130
1460,FL,Miami-Dade (County),DR,2003-04-25,SEVERE STORMS AND TORNADOES,1,14538921,6659


## Other Resources
 - [OpenFEMA Tutorials on GitHub](https://github.com/FEMA/openfema-samples/tree/master/analysis-examples)
 - [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api)